In [1]:
import transformers
print(transformers.__version__)

4.5.0


In [2]:
from datasets import load_dataset

In [3]:
datasets = load_dataset("natural_questions",cache_dir='/storage/datset')

Using custom data configuration default
Reusing dataset natural_questions (/storage/datset/natural_questions/default/0.0.2/867dbbaf9137c1b83ecb19f5eb80559e1002ea26e702c6b919cfa81a17a8c531)


In [4]:
import re


def get_nq_tokens(simplified_nq_example, i):
    '''Returns list of blank separated tokens.'''

    if "context" not in simplified_nq_example:
        raise ValueError("`get_nq_tokens` should be called on a simplified NQ"
                     "example that contains the `document_text` field.")

    return simplified_nq_example["context"][i].split(" ")


def simplify_nq_example(nq_example):
  r"""Returns dictionary with blank separated tokens in `document_text` field.
  Removes byte offsets from annotations, and removes `document_html` and
  `document_tokens` fields. All annotations in the ouput are represented as
  [start_token, end_token) offsets into the blank separated tokens in the
  `document_text` field.
  WARNING: Tokens are separated by a single blank character. Do not split on
    arbitrary whitespace since different implementations have different
    treatments of some unicode characters such as \u180e.
  Args:
    nq_example: Dictionary containing original NQ example fields.
  Returns:
    Dictionary containing `document_text` field, not containing
    `document_tokens` or `document_html`, and with all annotations represented
    as [`start_token`, `end_token`) offsets into the space separated sequence.
  """

  def _clean_token(token):
    """Returns token in which blanks are replaced with underscores.
    HTML table cell openers may contain blanks if they span multiple columns.
    There are also a very few unicode characters that are prepended with blanks.
    Args:
      token: Dictionary representation of token in original NQ format.
    Returns:
      String token.
    """
    return re.sub(u" ", "_", token)

  tokens = [[_clean_token(t) for t in example['tokens']['token']] for example in nq_example['document']]
  text = [" ".join(t) for t in tokens]
  #text = " ".join([_clean_token(t) for t in nq_example["document_tokens"]])

  def _remove_html_byte_add_char_offsets(span, tokens):
    if "start_byte" in span:
      del span["start_byte"]

    if "end_byte" in span:
      del span["end_byte"]

    start_tok = span["start_token"]
    end_tok = span["end_token"]

    chars = [len(s) for s in tokens]


    if not isinstance(start_tok,list):
        init , end = -1, -1
        if start_tok != -1:
            init = sum(chars[:start_tok])+ start_tok
            end = init + sum(chars[start_tok:end_tok])+ end_tok - start_tok-1
        span['start_char']=init
        span['end_char']=end
        '''
        init, end =[],[]
        for i in range(len(start_tok)):
            s = sum(chars[:start_tok[i]])+ start_tok[i]
            init.append(s)
            end.append(s + sum(chars[start_tok[i]:end_tok[i]])+ end_tok[i] - start_tok[i]-1)
        '''



    return span

  def _clean_annotation(annotation, tokens):
    for i,(la, t) in enumerate(zip(annotation, tokens)):
        annotation[i]["long_answer"]=_remove_html_byte_add_char_offsets(la["long_answer"][0], t)
        del annotation[i]["short_answers"]
    '''
    annotation["long_answer"] = [
        _remove_html_byte_add_char_offsets(la["long_answer"][0], t) for (la, t) in zip(annotation, tokens)
    ]
    annotation["short_answers"] = [
        _remove_html_byte_add_char_offsets(sa, tokens) for sa in annotation["short_answers"]
    ]
    '''
    
    return annotation


  simplified_nq_example = {
      "question": [example["text"] for example in nq_example["question"]],
      "id": nq_example["id"],
      #"document_url": [example["url"] for example in nq_example["document"]],
      "context": text,
      "annotations": _clean_annotation(nq_example["annotations"], tokens)
  }

  for i in range(len(nq_example['document'])):
    if len(get_nq_tokens(simplified_nq_example, i)) != len(
        nq_example['document'][i]['tokens']['token']):
      raise ValueError("Incorrect number of tokens.")

  return simplified_nq_example

In [5]:
from transformers import PreTrainedTokenizerFast, AutoTokenizer
model_pretrain = "bert-large-cased-whole-word-masking-finetuned-squad"#'roberta-large'   #"bert-large-uncased" 
#tokenizer =  PreTrainedTokenizerFast.from_pretrained(model_pretrain)
tokenizer = AutoTokenizer.from_pretrained(
    model_pretrain,
    use_fast=True,
)
print(tokenizer.vocab_size)
#from transformers import TransfoXLTokenizer, TransfoXLModel
#tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')
#model_pretrain = TransfoXLModel.from_pretrained('transfo-xl-wt103')

max_length = 512
doc_stride = 128
batch_size = 1
pad_on_right = tokenizer.padding_side == "right"
print(tokenizer.pad_token,pad_on_right)
print(tokenizer.encode('<P> apple </P>'))
tokenizer.add_tokens(['<P>','</P>', '<Table>','</Table>','<Li>','</Li>','<Th>','</Th>','<Td>','</Td>','Ul','/Ul'],special_tokens=True)
print(len(tokenizer))
#tokenizer.add_special_tokens({'cls_token': '[CLS]'})
#tokenizer.add_special_tokens({'sep_token': '[SEP]'})
print(tokenizer.encode('<P> apple </P>'))

28996
[PAD] True
[101, 133, 153, 135, 12075, 133, 120, 153, 135, 102]
29008
[101, 28996, 12075, 28997, 102]


In [33]:
test = simplify_nq_example(datasets['train'][2:3])
start_char = test["annotations"][0]['long_answer']['start_char']
end_char = test["annotations"][0]['long_answer']['end_char']
ans = test['context'][0][start_char:end_char+1]


In [6]:
# Training preprocessing
def prepare_train_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    examples = simplify_nq_example(examples)

    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        start_char = examples["annotations"][sample_index]["long_answer"]["start_char"]
        # If no answers are given, set the cls_index as answer.
        if start_char == -1:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            end_char = examples["annotations"][sample_index]["long_answer"]["end_char"]

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

# Validation preprocessing
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    examples = simplify_nq_example(examples)
    
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)           
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [26]:
#prepare_train_features(datasets['train'][:2])
'''
examples = simplify_nq_example(datasets['train'][:4])
tokenized_examples = tokenizer(
    examples["question_text" if pad_on_right else "document_text"],
    examples["document_text" if pad_on_right else "question_text"],
    truncation="only_second" if pad_on_right else "only_first",
    max_length=max_length,
    stride=doc_stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
)

tokenizer = AutoTokenizer.from_pretrained(
    model_bert,
    use_fast=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_pretrain,
    use_fast=True,
)


tokenized_examples = prepare_train_features(datasets['train'][3:4])
print(tokenized_examples['start_positions'])
print(tokenized_examples['end_positions'])
print(tokenizer.decode(tokenized_examples["input_ids"][0][41:298+1]))
'''
tokenized_examples = prepare_validation_features(datasets['validation'][3:4])
for key in tokenized_examples:
    print(key)
print(tokenized_examples['example_id'][0:2])

Using pad_token, but it is not set yet.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
'''
input_ids
token_type_ids
attention_mask
start_positions
end_positions
'''
import random
import os
from datasets import concatenate_datasets

sample_size = 4000
pre_size = 100000
total_size = 220000
save_dir = os.path.join('/storage','BERT_SQUAD_TOK')
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
data_list = []
for i in range(pre_size, total_size,sample_size):
    test_train = datasets['train'].select(range(i,min(i+sample_size,len(datasets['train'])))) #307373
    tokenized_test = test_train.map(prepare_train_features, batched=True, remove_columns=test_train.column_names)
    print('i: ',i ,'sample_size', sample_size, 'padded sentences',len(tokenized_test))

    neg = tokenized_test.filter(lambda example: example['start_positions']==0)
    pos = tokenized_test.filter(lambda example: example['start_positions']!=0)
    print('neg: ', len(neg),'pos: ', len(pos))

    neg_down = neg.select(random.sample(range(0,len(neg)-1), len(pos)))
    data_list.extend([neg_down, pos])
total_train = concatenate_datasets(data_list)
total_train.save_to_disk(os.path.join(save_dir, "train_{}_{}".format(total_size-pre_size, len(total_train))))


i:  100000 sample_size 4000 padded sentences 132589



neg:  130962 pos:  1627



i:  104000 sample_size 4000 padded sentences 132822



neg:  131146 pos:  1676


In [43]:
# save validation examples
sample_size = 2000
pre_size = 0
total_size = len(datasets['validation'])
data_list = []
dir_name = 'roberta'
for i in range(pre_size, total_size,sample_size):
    test_val = datasets['validation'].select(range(i,min(i+sample_size,total_size))) #7830
    tokenized_val = test_val.map(simplify_nq_example, batched=True, remove_columns=test_val.column_names)

    data_list.extend([tokenized_val])
total_val = concatenate_datasets(data_list)
total_val.save_to_disk("/storage/{}/val_example".format(dir_name))

In [ ]:
# save validation features
import random
sample_size = 2000
pre_size = 0
total_size = len(datasets['validation'])
data_list = []
dir_name = 'roberta'
for i in range(pre_size, total_size,sample_size):
    test_val = datasets['validation'].select(range(i,min(i+sample_size,total_size))) #7830
    tokenized_val = test_val.map(prepare_validation_features, batched=True, remove_columns=test_val.column_names)
    print('i: ',i ,'sample_size', sample_size, 'padded sentences',len(tokenized_val))
    '''
    neg = tokenized_val.filter(lambda example: example['start_positions']==0)
    pos = tokenized_val.filter(lambda example: example['start_positions']!=0)
    print('neg: ', len(neg),'pos: ', len(pos))

    neg_down = neg.select(random.sample(range(0,len(neg)-1), len(pos)))
    data_list.extend([neg_down, pos])
    '''
    data_list.extend([tokenized_val])
total_val = concatenate_datasets(data_list)
total_val.save_to_disk("/storage/{}/val".format(dir_name))

In [ ]:
from datasets import concatenate_datasets
from datasets import load_from_disk
import os
save_dir = '/storage/BERT_SQUAD_TOK'
train100000=load_from_disk(os.path.join(save_dir, "train_{}_{}".format(100000,82522)))
train120000=load_from_disk(os.path.join(save_dir, "train_{}_{}".format(120000,95356)))
total_train = concatenate_datasets([train100000, train120000])
total_train.save_to_disk(os.path.join(save_dir, "train_{}_{}".format(220000,len(total_train))))
#valid = load_from_disk("/storage/val")

In [4]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator


model = AutoModelForQuestionAnswering.from_pretrained(model_pretrain)
args = TrainingArguments(
    "/storage/model/base",
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
)

data_collator = default_data_collator
trainer = Trainer(
    model,
    args,
    train_dataset=train,
    eval_dataset=valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-large-uncased

In [5]:
trainer.train()

Epoch,Training Loss,Validation Loss


RuntimeError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 7.94 GiB total capacity; 7.40 GiB already allocated; 13.00 MiB free; 7.41 GiB reserved in total by PyTorch)

In [21]:
import torch
del trainer
torch.cuda.empty_cache()

In [56]:
total_train.save_to_disk("/storage/{}/train_{}_{}".format(dir_name, total_size, len(total_train)))

In [60]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-9ehcxkg9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |################################| 3.2 MB 22.6 MB/s eta 0:00:01
  Created wheel for transformers: filename=transformers-4.5.0.dev0-py3-none-any.whl size=2064484 sha256=e89640567f4dc68b29d31f7dd9892eee6f4a0558a59cbf98b758ad6499e2949f
  Stored in directory: /tmp/pip-ephem-wheel-cache-7tzu6s06/wheels/5a/0a/d0/eb8d0ea1d7d02156f8675d6e5dfa52c03601cbe377290db8dc
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.3
    Uninstalling tokenizers-0.9.3:
      Successfully uninstalled tokenizers-0.9.3
  Attempting uninstall: transformers
    Found existing installation: transformers 3.5.1
    Uninstalling transformers-3.5.1:
      Successfully uninstalled transformers-3.5.1
You should consider upgrading via the '/usr/bin/p

In [7]:
import transformers
print(transformers.__version__)

3.5.1


In [16]:
print(tokenizer.sep_token_id)

2
